In [22]:
import numpy as np


from pathlib import Path
import os
cwd=os.getcwd()

import pymc3 as pm
import theano
import theano.tensor as tt
import integration as inn
import background as bg


import arviz as az
az.style.use("arviz-darkgrid")

print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.9.3


In [2]:
path_data = cwd + '/data/SN.txt'
data_file = Path(path_data)
if data_file.is_file():
    data = np.genfromtxt(path_data)
    z_obs = data[:,0]
    mu_obs = data[:,1]
    dmu_obs = data[:,2]
    print("SN data are loaded")
    
else:
    print("SN files do not exist")
    sys.exit(0) 

SN data are loaded


In [9]:
num = 2
@theano.compile.ops.as_op(itypes=[tt.dscalar for i in range(num)],otypes=[tt.dvector])
def mu_the_pymc(H0,om):
        return bg.mu_the(z_obs,H0,om)


def log_likelihood(H0,om):
    mu = mu_the_pymc(H0,om)
    x = (mu - mu_obs)/dmu_obs 
    return -0.5*pm.math.dot(x,x)


with pm.Model() as model:
    H0 = pm.Uniform('H0', 60., 80.)
    om = pm.Uniform('om', 0.1, .4)

    #like = pm.Potential('like', log_likelihood(H0,om))
    y = pm.Normal('y', mu=mu_the_pymc(H0,om), sd=dmu_obs, observed=mu_obs)

    trace = pm.sample_smc(1000,random_seed=22,parallel=True)

Initializing SMC sampler...
Sampling 2 chains in 2 jobs
Stage:   0 Beta: 0.002
Stage:   1 Beta: 0.014
Stage:   2 Beta: 0.060
Stage:   3 Beta: 0.205
Stage:   4 Beta: 0.657
Stage:   5 Beta: 1.000


In [21]:
trace.report.log_marginal_likelihood
az_trace = az.from_pymc3(trace)
ax = az.plot_trace(az_trace, compact=True)

array([2.65614447e-14, 3.06770164e-14])